In [ ]:
import pandas as pd
import ast
import numpy as np
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
tf.disable_v2_behavior()

from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from google.colab import drive


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
drive.mount('/content/drive')
dataSetFolder = '/content/drive/My Drive/movies_data'

Mounted at /content/drive


In [ ]:
movies = pd. read_csv(dataSetFolder+'/movies_metadata.csv')
extlinks = pd.read_csv(dataSetFolder+'/links_small.csv')
ratings = pd.read_csv(dataSetFolder+'/ratings_small.csv')
extlinks.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [ ]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
credits = pd.read_csv(dataSetFolder+'/credits.csv')
keywords = pd.read_csv(dataSetFolder+'/keywords.csv')
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [ ]:
credits['cast'][0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [ ]:
movies = movies.drop([19730, 29503, 35587])
extlinks = extlinks[extlinks['tmdbId'].notnull()]['tmdbId'].astype('int')
movies['genres'] = movies['genres'].fillna('[]').apply(ast.literal_eval).apply(lambda genres: [genre['name'] for genre in genres] if isinstance(genres, list) else [])
movies['id'] = movies['id'].astype('int')
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
movies = movies.merge(credits, on='id')
movies = movies.merge(keywords, on='id')
movies_filtered = movies[movies['id'].isin(extlinks)]
#movies_filtered.head()
#movies_filtered.shape

In [ ]:
movies_filtered.head()
#movies_filtered.shape

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [ ]:
def getDirector(crew):
  for person in crew:
    if person['job'] == 'Director':
          return person['name']
  return np.nan
movies_filtered['cast'] = movies_filtered['cast'].apply(ast.literal_eval) #since the cast column has disctionary, we use the ast.literal_eval to convert the column values into dictionaries
movies_filtered['crew'] = movies_filtered['crew'].apply(ast.literal_eval) #since the crew column has disctionary, we use the ast.literal_eval to convert the column values into dictionaries
movies_filtered['keywords'] = movies_filtered['keywords'].apply(ast.literal_eval) #since the keywords column has disctionary, we use the ast.literal_eval to convert the column values into dictionaries
movies_filtered['cast_size'] = movies_filtered['cast'].apply(lambda x: len(x)) #storing the number of cast members (who act in the movie)
movies_filtered['crew_size'] = movies_filtered['crew'].apply(lambda x: len(x)) #storing the number of crew members 
 
movies_filtered['director'] = movies_filtered['crew'].apply(getDirector) #creating the column for the director name 
movies_filtered['cast']  = movies_filtered['cast'].apply(lambda cast: [person['name'] for person in cast] if isinstance(cast, list) else []) #the cast column now is a list of names 
movies_filtered['cast'] = movies_filtered['cast'].apply(lambda cast: cast[:3] if len(cast) >=3 else cast) #we just load only 3 cast members 
movies_filtered['keywords'] = movies_filtered['keywords'].apply(lambda keywords: [keyword['name'] for keyword in keywords] if isinstance(keywords, list) else []) #
movies_filtered['director'] = movies_filtered['director'].astype('str').apply(lambda name: str.lower(name.replace(" ", "")))
movies_filtered['director'] = movies_filtered['director'].apply(lambda name: [name, name, name])
keywords = movies_filtered.apply(lambda movie: pd.Series(movie['keywords']),axis=1).stack().reset_index(level=1, drop=True)
keywords = keywords.value_counts()
keywords = keywords[keywords>1]
keywords.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pan

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
dtype: int64

In [ ]:
movies_filtered.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords,cast_size,crew_size,director
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[Tom Hanks, Tim Allen, Don Rickles]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[jealousy, toy, boy, friendship, friends, riva...",13,106,"[johnlasseter, johnlasseter, johnlasseter]"
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[board game, disappearance, based on children'...",26,16,"[joejohnston, joejohnston, joejohnston]"
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[Walter Matthau, Jack Lemmon, Ann-Margret]","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[fishing, best friend, duringcreditsstinger, o...",7,4,"[howarddeutch, howarddeutch, howarddeutch]"
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[Whitney Houston, Angela Bassett, Loretta Devine]","[{'credit_id': '52fe44779251416c91011acb', 'de...","[based on novel, interracial relationship, sin...",10,10,"[forestwhitaker, forestwhitaker, forestwhitaker]"
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[Steve Martin, Diane Keaton, Martin Short]","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[baby, midlife crisis, confidence, aging, daug...",12,7,"[charlesshyer, charlesshyer, charlesshyer]"


In [ ]:
len(keywords)

6709

In [ ]:
def filterKeywords(list):
  words = []
  for word in list:
    if word in keywords:
      words.append(word)
  return words

# in the following code we try to stem the keywords, lower case all the words and create a data set that is a simple string
# which consists of the keywords use to describe movies
# cast name
# director's name 
# genres 
movies_filtered['keywords'] = movies_filtered['keywords'].apply(filterKeywords)
movies_filtered['keywords'] = movies_filtered['keywords'].apply(lambda movieKeywords: [SnowballStemmer('english').stem(word) for word in movieKeywords]) #stemming the word in the keywords
movies_filtered['keywords'] = movies_filtered['keywords'].apply(lambda movieKeywords: [str.lower(word.replace(" ", "")) for word in movieKeywords])# removing the spaces from the words

movies_filtered['document'] = movies_filtered['keywords'] + movies_filtered['cast'] + movies_filtered['director'] + movies_filtered['genres']
movies_filtered['document'] = movies_filtered['document'].apply(lambda x: ' '.join(x))
movies_filtered['document'] 

#count vectorixer creates a vector of word for each sentence
count_vector = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count_vector.fit_transform(movies_filtered['document'] )
# we then calculate the cosine similarity based on the couknt vecotrs obtained in the above step
similarity = cosine_similarity(count_matrix, count_matrix)

movies_filtered = movies_filtered.reset_index()
titles = movies_filtered['title']
indices = pd.Series(movies_filtered.index, index=movies_filtered['title'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [ ]:
count_matrix[0]

<1x117213 sparse matrix of type '<class 'numpy.int64'>'
	with 38 stored elements in Compressed Sparse Row format>

In [ ]:
movies_filtered['document'][0] 

'jealousi toy boy friendship friend rivalri boynextdoor newtoy toycomestolif Tom Hanks Tim Allen Don Rickles johnlasseter johnlasseter johnlasseter Animation Comedy Family'

In [ ]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    print(sim_scores)
    return titles.iloc[movie_indices]

In [ ]:
content_based_recommendations = get_recommendations('Batman Begins').head(10)

[(6981, 0.4834981269087812), (8031, 0.44566881162492417), (6623, 0.32398516128391164), (2085, 0.25521383116292545), (7648, 0.220898973602667), (4145, 0.2168454370598277), (3381, 0.18793271211851734), (8613, 0.15910118196788303), (1134, 0.15353487969536544), (6645, 0.14930466958822783), (1260, 0.14074531720101102), (7513, 0.13734032069989463), (7561, 0.13734032069989463), (1388, 0.1329791937083694), (7659, 0.13253938416160022), (9024, 0.12785688039610243), (7584, 0.1268540658512312), (3749, 0.125481281000767), (8925, 0.12156613477096613), (3454, 0.1201727806124078), (6937, 0.1201727806124078), (8725, 0.11968127433753244), (8001, 0.11781278592142241), (2754, 0.11720509341348334), (7333, 0.11720509341348334), (1463, 0.11444694015889996), (4427, 0.11444694015889996), (3647, 0.11197850219117088), (2752, 0.11187477034658963), (3597, 0.11187477034658963)]


In [ ]:
content_based_recommendations

6981          The Dark Knight
8031    The Dark Knight Rises
6623             The Prestige
2085                Following
7648                Inception
4145                 Insomnia
3381                  Memento
8613             Interstellar
1134           Batman Returns
6645              Harsh Times
Name: title, dtype: object

In [ ]:
ratings.count()

userId       100004
movieId      100004
rating       100004
timestamp    100004
dtype: int64

In [ ]:
ratings.userId = ratings.userId.astype(str).astype(int)
ratings.movieId = ratings.movieId.astype(str).astype(int)
ratings.rating = ratings.rating.astype(str).astype(float)
ratings.timestamp = pd.to_datetime(ratings.timestamp.astype(int), unit='s')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,2009-12-14 02:52:24
1,1,1029,3.0,2009-12-14 02:52:59
2,1,1061,3.0,2009-12-14 02:53:02
3,1,1129,2.0,2009-12-14 02:53:05
4,1,1172,4.0,2009-12-14 02:53:25


In [ ]:
len(movies_filtered)
movies_filtered['List Index']=movies_filtered.index.astype(str).astype(int)

In [ ]:
data_combined=pd.merge(movies_filtered, ratings, left_on='id' , right_on='movieId')

In [ ]:
data_combined = data_combined[['movieId', 'List Index', 'userId', 'rating']]
data_combined['List Index']=data_combined['List Index'].astype(str).astype(int)
data_combined.head()

,movieId,List Index,userId,rating
0,949,5,23,3.5
1,949,5,102,4.0
2,949,5,232,2.0
3,949,5,242,5.0
4,949,5,263,3.0


In [ ]:
user_group = data_combined.groupby('userId')

In [ ]:
user_group.head()

,movieId,List Index,userId,rating
0,949,5,23,3.5
1,949,5,102,4.0
2,949,5,232,2.0
3,949,5,242,5.0
4,949,5,263,3.0
...,...,...,...,...
31323,5816,7180,179,5.0
31539,1961,7235,374,4.0
31890,51540,7913,40,4.5
31975,85414,8190,526,3.0


In [ ]:
data_combined.drop('List Index', inplace=True, axis=1)

In [ ]:
data_combined.head()

,movieId,userId,rating
0,949,23,3.5
1,949,102,4.0
2,949,232,2.0
3,949,242,5.0
4,949,263,3.0


In [ ]:
data_combined.to_csv(dataSetFolder+'/filtered.csv',index=False)

In [ ]:
ks = pd.read_csv(dataSetFolder+'/filtered.csv')

In [ ]:
ks.head()

,movieId,userId,rating
0,949,23,3.5
1,949,102,4.0
2,949,232,2.0
3,949,242,5.0
4,949,263,3.0
